Import modules

In [1]:
import configparser
import os
from pyspark.sql import SparkSession
import shutil
from datetime import datetime

In [2]:
#load config info
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


# Create spark session

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

# Load song data from S3

In [5]:
df = spark.read.json('s3://udacity-dend/song_data/*/*/Z/*.json')

Py4JJavaError: An error occurred while calling o67.json.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:547)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:545)
	at scala.collection.TraversableLike$$anonfun$fl

In [ ]:
df.printSchema()

# Create a new view to select song data and write to parquet 

In [56]:
df.createOrReplaceTempView("song_data")
songs_table = spark.sql("""
    SELECT song_id, title, artist_id, year, duration
    FROM song_data
""")
import shutil
shutil.rmtree("./data/output/songs_table", ignore_errors=True)

songs_table.write.partitionBy("year", "artist_id").parquet("./data/output/songs_table")
songs_table.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOYMRWW12A6D4FAB14|The Moon And I (O...|ARKFYS91187B98E58F|   0| 267.7024|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
|SOQHXMF12AB0182363|     Young Boy Blues|ARGSJW91187B9B1D6B|   0|218.77506|
|SOCIWDW12A8C13D406|           Soul Deep|ARMJAGH1187FB546F3|1969|148.03546|
|SONHOTT12A8C13493C|     Something Girls|AR7G5I41187FB4CE6C|1982|233.40363|
|SOMZWCG12A8C13C480|    I Didn't Mean To|ARD7TVE1187B99BFB1|   0|218.93179|
|SOUDSGM12AC9618304|Insatiable (Instr...|ARNTLGG11E2835DDB9|   0|266.39628|
|SOMJBYD12A6D4F8557|Keepin It Real (S...|ARD0S291187B9B7BF5|   0|114.78159|
|SOXVLOJ12AB0189215|     Amor De Cabaret|ARKRRTF1187B9984DA|   0|177.47546|
|SOIAZJW12AB01853F1|          Pink World|AR8ZCNI1187B9A069B|1984|269.81832|
|SOFSOCN12A8

# write artist table

In [57]:
artists_table = spark.sql("""
    SELECT artist_id, min(artist_name) as artist_name, min(artist_location) as artist_location, 
    min(artist_latitude) as artist_latitude, min(artist_longitude) as artist_longitude
    FROM song_data GROUP BY artist_id""")
shutil.rmtree("./data/output/artists_table", ignore_errors=True)
artists_table.write.parquet("./data/output/artists_table")
artists_table.show()

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARNTLGG11E2835DDB9|                 Clp|                    |           null|            null|
|ARKRRTF1187B9984DA|    Sonora Santanera|                    |           null|            null|
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|           null|            null|
|AR7G5I41187FB4CE6C|            Adam Ant|     London, England|           null|            null|
|ARKFYS91187B98E58F|Jeff And Sheri Ea...|                    |           null|            null|
|AR8ZCNI1187B9A069B|    Planet P Project|                    |           null|            null|
|ARGSJW91187B9B1D6B|        JennyAnyKind|      North Carolina|       35.21962|       -80.01955|
|ARD0S291187B9B7BF5|             Rated R

In [58]:
log_data = "./data/" + "log_data/*/*/*.json"
dfl = spark.read.json(log_data)
dfl.createOrReplaceTempView("log_data")
dfl = spark.sql("""SELECT * FROM log_data WHERE page='NextSong'""")
dfl.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [59]:
users_table = spark.sql("""
    SELECT userId as user_id, min(firstName) as first_name, min(lastName) as last_name, min(gender) as gender, min(level) as level
    FROM log_data GROUP BY userId
""")
shutil.rmtree("./data/output/users_table",ignore_errors=True)
users_table.write.parquet("./data/output/users_table")
users_table.show()

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     51|      Maia|    Burke|     F| free|
|      7|    Adelyn|   Jordan|     F| free|
|     15|      Lily|     Koch|     F| free|
|     54|     Kaleb|     Cook|     M| free|
|    101|    Jayden|      Fox|     M| free|
|     11| Christian|   Porter|     F| free|
|     29|Jacqueline|    Lynch|     F| free|
|     69|  Anabelle|  Simpson|     F| free|
|     42|    Harper|  Barrett|     M| paid|
|     73|     Jacob|    Klein|     M| paid|
|     64|    Hannah|  Calhoun|     F| free|
|      3|     Isaac|   Valdez|     M| free|
|     30|     Avery|  Watkins|     F| paid|
|     34|    Evelin|    Ayala|     F| free|
|     59|      Lily|   Cooper|     F| free|
|      8|    Kaylee|  Summers|     F| free|
|     28|  Brantley|     West|     M| free|
|     85|   Kinsley|    Young|     F| free|
|     16|     Rylan|   George|     M| free|
|     35|     Molly|   Taylor|  

In [60]:
import  pyspark.sql.functions as F
dfl = dfl.withColumn('timestamp', F.to_timestamp(dfl['ts']/1000))
dfl = dfl.withColumn('datetime', F.to_date('timestamp'))
dfl.createOrReplaceTempView("log_data")

In [61]:
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek
time_table = spark.sql("""
    SELECT ts as start_time, hour(timestamp) as hour, dayofmonth(timestamp) as day, weekofyear(timestamp) as week,
    month(timestamp) as month, year(timestamp) as year, dayofweek(timestamp) as weekday
    FROM log_data
""")
shutil.rmtree("./data/output/time_table",ignore_errors=True)
time_table.write.partitionBy("year", "month").parquet("./data/output/time_table")
time_table.show()

+-------------+----+---+----+-----+----+-------+
|   start_time|hour|day|week|month|year|weekday|
+-------------+----+---+----+-----+----+-------+
|1542241826796|   0| 15|  46|   11|2018|      5|
|1542242481796|   0| 15|  46|   11|2018|      5|
|1542242741796|   0| 15|  46|   11|2018|      5|
|1542253449796|   3| 15|  46|   11|2018|      5|
|1542260935796|   5| 15|  46|   11|2018|      5|
|1542261224796|   5| 15|  46|   11|2018|      5|
|1542261356796|   5| 15|  46|   11|2018|      5|
|1542261662796|   6| 15|  46|   11|2018|      5|
|1542262057796|   6| 15|  46|   11|2018|      5|
|1542262233796|   6| 15|  46|   11|2018|      5|
|1542262434796|   6| 15|  46|   11|2018|      5|
|1542262456796|   6| 15|  46|   11|2018|      5|
|1542262679796|   6| 15|  46|   11|2018|      5|
|1542262728796|   6| 15|  46|   11|2018|      5|
|1542262893796|   6| 15|  46|   11|2018|      5|
|1542263158796|   6| 15|  46|   11|2018|      5|
|1542263378796|   6| 15|  46|   11|2018|      5|
|1542265716796|   7|

In [78]:
output_data = "./data/output/"
song_df = spark.read.parquet(output_data + "songs_table")
song_df.createOrReplaceTempView("song_view")
artist_df = spark.read.parquet(output_date + "artists_table")
artist_df.createOrReplaceTempView("artist_view")
artist_df.show()

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|           null|            null|
|ARGSJW91187B9B1D6B|        JennyAnyKind|      North Carolina|       35.21962|       -80.01955|
|ARMJAGH1187FB546F3|        The Box Tops|         Memphis, TN|       35.14968|       -90.04892|
|AR7G5I41187FB4CE6C|            Adam Ant|     London, England|           null|            null|
|ARD7TVE1187B99BFB1|              Casual|     California - LA|           null|            null|
|ARKFYS91187B98E58F|Jeff And Sheri Ea...|                    |           null|            null|
|AR8ZCNI1187B9A069B|    Planet P Project|                    |           null|            null|
|ARKRRTF1187B9984DA|    Sonora Santanera

In [79]:
song_df.count()

11

In [80]:
artist_df.count()

11

In [82]:
song_plays_table = spark.sql("""
    SELECT monotonically_increasing_id() as songplay_id, ts as start_time, userId as user_id, level, sv.song_id, av.artist_id, sessionId as session_id,
    location, userAgent as user_agent FROM log_data ld
    LEFT JOIN song_view sv ON ld.song=sv.title
    LEFT JOIN artist_view av ON ld.artist=av.artist_name""")
song_plays_table.show()

+-----------+-------------+-------+-----+-------+---------+----------+--------------------+--------------------+
|songplay_id|   start_time|user_id|level|song_id|artist_id|session_id|            location|          user_agent|
+-----------+-------------+-------+-----+-------+---------+----------+--------------------+--------------------+
|          0|1542241826796|     26| free|   null|     null|       583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|
|          1|1542242481796|     26| free|   null|     null|       583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|
|          2|1542242741796|     26| free|   null|     null|       583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|
|          3|1542253449796|     61| free|   null|     null|       597|Houston-The Woodl...|"Mozilla/5.0 (Mac...|
|          4|1542260935796|     80| paid|   null|     null|       602|Portland-South Po...|"Mozilla/5.0 (Mac...|
|          5|1542261224796|     80| paid|   null|     null|       602|Portland-South Po...|"Mozi

In [ ]:
shutil.rmtree(output_data + "songplays_table",ignore_errors=True)
song_plays_table.write.partitionBy("user_id").parquet(output_data + "songplays_table")

In [7]:
import shutil
shutil.rmtree("./data/output/artists_table", ignore_errors=True)
shutil.rmtree("./data/output/songs_table", ignore_errors=True)
shutil.rmtree("./data/output/songplays_table", ignore_errors=True)
shutil.rmtree("./data/output/time_table", ignore_errors=True)
shutil.rmtree("./data/output/users_table", ignore_errors=True)
shutil.rmtree("./songs_table", ignore_errors=True)